<font size="6"> **Portfolio Optimization** </font>

In [1]:
%load_ext autoreload
%autoreload 2
%run ../nb_config.py

In [2]:
RND_SEED = 123
RISK_MODEL_VERSION = 'v01'
MLALPHA_MODEL_VERSION = 'v01'
OPT_MODEL_VERSION = 'v01'
print('Risk Model version:', RISK_MODEL_VERSION, 'ML-Alpha model version:', MLALPHA_MODEL_VERSION)

Risk Model version: v01 ML-Alpha model version: v01


In [3]:
import os

import numpy as np
import scipy
import pickle

from src import utils
from src.load_data import io_utils
from src.mle_quant_utils import mle_utils

In [4]:
cfg = utils.read_conf()

INFILE1 = cfg['output']['alpha_factors_ml']['preds']
INFILE2 = cfg['output']['interim']['all_factors']
INFILE3 = cfg['output']['interim']['pricing']
INFILE4 = cfg['output']['interim']['volume']

OUTFILE1 = cfg['output']['opt_portfolio']['pnl_results']
OUTFILE2 = cfg['output']['opt_portfolio']['model']
OUTFILE3 = cfg['output']['opt_portfolio']['model_metadata']

In [ ]:
OUTPATH1 =  os.path.join(io_utils.processed_path, cfg['output']['opt_portfolio']['folder'], OPT_MODEL_VERSION, "")
OUTPATH2 =  os.path.join(io_utils.models_path, cfg['output']['opt_portfolio']['folder'], OPT_MODEL_VERSION, "")
print(os.path.isdir(OUTPATH1), os.path.isdir(OUTPATH2))

In [6]:
INPATH_RISK = os.path.join(io_utils.processed_path, cfg['output']['risk_factors']['folder'], RISK_MODEL_VERSION, "")
INPATH_ALPHA = os.path.join(io_utils.processed_path, cfg['output']['alpha_factors_ml']['folder'], MLALPHA_MODEL_VERSION, "")

print(os.path.isdir(INPATH_RISK), os.path.isdir(INPATH_ALPHA))

True True


In [ ]:
TAU = cfg['models']['alpha_ml']['tau']
target_col = cfg['models']['alpha_ml']['target_col']

splits = cfg['models']['alpha_ml']['splits']
split_col = cfg['models']['alpha_ml']['split_col']

transcost_adv_win = int(cfg['backtesting']['transcost_adv_win'])
transcost_adv_impute = float(cfg['backtesting']['transcost_adv_impute'])

# Load Data

## Risk Model Data

In [7]:
risk_model = {}
risk_files = ['idiosyncratic_var_vector', 'factor_var_vector', 'factor_betas']
for file in risk_files:
    risk_model[file] = pd.read_csv(INPATH_RISK + file + '.csv', index_col=0)
risk_model['factor_returns'] = pd.read_csv(INPATH_RISK + 'factor_returns.csv', parse_dates=['date'], index_col='date')

In [8]:
pd.concat([risk_model['factor_betas'].head(2), risk_model['factor_betas'].tail(2)], axis=0)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
A,-0.042628,0.014983,0.012869,0.003801,-0.022948,0.010030,-0.004048,-0.006718,-0.034471,0.024555,-0.017108,0.018875,0.025266,0.014909,-0.026406,0.033897,-0.029173,-0.030546,-0.026341,-0.012199
AAL,-0.060402,0.258601,-0.037516,0.144527,0.296615,-0.080756,0.113555,-0.024766,-0.161712,-0.035469,0.043421,0.100238,-0.014771,-0.026967,0.015362,0.025396,-0.013467,-0.056342,0.067339,0.092211
ZION,-0.061633,-0.007099,0.011137,0.132754,0.003718,0.084665,-0.047824,0.025166,0.045630,-0.035794,0.018539,-0.011147,-0.012005,-0.025919,0.020266,0.001111,-0.002798,0.019725,0.009674,0.020515
ZTS,-0.009068,0.018116,-0.007967,-0.024529,0.004524,0.023557,-0.012210,-0.012917,-0.026895,0.001144,-0.021353,-0.035603,-0.026044,-0.007709,-0.003599,-0.006892,0.021980,0.005291,0.002137,-0.013470


In [9]:
pd.concat([risk_model['factor_returns'].head(2), risk_model['factor_returns'].tail(2)], axis=0)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
date,,,,,,,,,,,,,,,,,,,,
2015-05-01 00:00:00+00:00,-0.231498,0.087009,-0.035185,-0.056721,0.006649,-0.014624,0.015049,-0.015601,-0.034371,0.031523,0.000828,-0.002096,0.017049,0.021042,0.019484,-0.058597,0.001738,-0.028662,0.026607,-0.074468
2015-05-04 00:00:00+00:00,-0.054719,0.007882,0.043880,0.020466,-0.049734,0.038801,-0.013299,0.006079,0.007780,-0.013627,0.000676,-0.017142,-0.028415,-0.011957,-0.016032,0.018925,-0.009592,0.018181,0.010830,-0.002648
2016-01-04 00:00:00+00:00,0.306592,-0.160879,0.008948,0.013268,0.031139,-0.058882,0.033337,-0.015162,-0.023391,-0.016096,0.097016,0.010075,-0.003835,0.040370,0.002952,-0.002508,-0.023565,0.020703,0.000560,-0.006553
2016-01-05 00:00:00+00:00,-0.029237,0.024917,0.105829,-0.051166,-0.021297,-0.008442,0.002659,-0.003693,0.021168,-0.058780,0.003526,-0.039558,-0.008585,-0.004922,-0.009306,0.006846,0.011186,0.006250,-0.037997,-0.006256


In [10]:
pd.concat([risk_model['factor_var_vector'].head(2), risk_model['factor_var_vector'].tail(2)], axis=0)

,0
0,13.101363
1,1.741896
18,0.201021
19,0.165043


In [11]:
pd.concat([risk_model['idiosyncratic_var_vector'].head(2), risk_model['idiosyncratic_var_vector'].tail(2)], axis=0)

,0
A,0.014894
AAL,0.029168
ZION,0.014803
ZTS,0.083648


## ML-Alpha

In [29]:
alpha_fac_df = pd.read_csv(INPATH_ALPHA + INFILE1, parse_dates=['date'], index_col=[0, 1])
pd.concat([alpha_fac_df.head(2), alpha_fac_df.tail(2)], axis=0)
#alpha_fac_df['date'] = alpha_fac_df['date'].dt.tz_localize('utc')
#alpha_fac_df.set_index(['date', 'asset'], inplace=True)

p_test  ml_alpha_raw  \
date                      asset                           
2015-05-26 00:00:00+00:00 A      0.000961      0.000961   
                          AAL    0.001474      0.001474   
2015-12-28 00:00:00+00:00 ZION   0.000835      0.000835   
                          ZTS   -0.000254     -0.000254   

                                 ml_alpha_sector_neutral  ml_alpha  \
date                      asset                                      
2015-05-26 00:00:00+00:00 A                     0.000819  0.441308   
                          AAL                   0.001575  0.921743   
2015-12-28 00:00:00+00:00 ZION                  0.000687  0.561338   
                          ZTS                  -0.000176 -0.219331   

                                 avg_combi_alpha  \
date                      asset                    
2015-05-26 00:00:00+00:00 A             0.483467   
                          AAL           1.197722   
2015-12-28 00:00:00+00:00 ZION          0.636962   
                          ZTS          -0.097619   

                                 Mean_Reversion_Sector_Neutral_Smoothed  \
date                      asset                                           
2015-05-26 00:00:00+00:00 A                                    1.534003   
                          AAL                                  1.692444   
2015-12-28 00:00:00+00:00 ZION                                 0.773366   
                          ZTS                                 -0.944433   

                                 Momentum_1YR_Smoothed  \
date                      asset                          
2015-05-26 00:00:00+00:00 A                  -1.233543   
                          AAL                 0.236520   
2015-12-28 00:00:00+00:00 ZION               -0.188031   
                          ZTS                 0.209727   

                                 Overnight_Sentiment_Smoothed  
date                      asset                                
2015-05-26 00:00:00+00:00 A                          1.149941  
                          AAL                        1.664201  
2015-12-28 00:00:00+00:00 ZION                       1.325552  
                          ZTS                        0.441851

In [30]:
alpha_fac_df.describe().T

,count,mean,std,min,25%,50%,75%,max
p_test,73655.0,-1.052683e-06,0.000545,-0.001751,-0.000363,-0.000001,0.000364,0.001753
ml_alpha_raw,73655.0,-1.052683e-06,0.000545,-0.001751,-0.000363,-0.000001,0.000364,0.001753
ml_alpha_sector_neutral,73655.0,1.089281e-21,0.000517,-0.001835,-0.000348,-0.000002,0.000348,0.001805
ml_alpha,73655.0,4.148169e-18,0.401922,-1.000000,-0.260191,-0.002555,0.264795,1.485041
avg_combi_alpha,73655.0,-2.353683e-03,0.512018,-1.711771,-0.358677,-0.004540,0.354455,1.706674
Mean_Reversion_Sector_Neutral_Smoothed,73655.0,3.935106e-04,0.997912,-1.728491,-0.862449,0.000000,0.862449,1.728491
Momentum_1YR_Smoothed,73655.0,-7.454560e-03,0.994537,-1.728439,-0.860603,-0.021696,0.853371,1.728439
Overnight_Sentiment_Smoothed,73655.0,1.495270e-18,1.000007,-1.728520,-0.866027,0.000000,0.866027,1.728520


In [14]:
all_factors_3yrs = pd.read_csv(io_utils.interim_path + INFILE2, parse_dates=['date'], index_col=[0,1])
#features_df['date'] = features_df['date'].dt.tz_localize('utc')
#features_df.set_index(['date', 'asset'], inplace=True)
split = all_factors_3yrs.pop(split_col)
features = all_factors_3yrs.columns.tolist()
pd.concat([all_factors_3yrs.head(2), all_factors_3yrs.tail(2)],axis=0)

Mean_Reversion_Sector_Neutral_Smoothed  \
date                      asset                                           
2013-01-03 00:00:00+00:00 A                                   -0.795709   
                          AAL                                  0.558847   
2015-12-28 00:00:00+00:00 ZION                                 0.773366   
                          ZTS                                 -0.944433   

                                 Momentum_1YR_Smoothed  \
date                      asset                          
2013-01-03 00:00:00+00:00 A                  -1.230279   
                          AAL                 1.713471   
2015-12-28 00:00:00+00:00 ZION               -0.188031   
                          ZTS                 0.209727   

                                 Overnight_Sentiment_Smoothed  adv_120d  \
date                      asset                                           
2013-01-03 00:00:00+00:00 A                         -1.301799  1.338573   
                          AAL                       -1.632765  1.139994   
2015-12-28 00:00:00+00:00 ZION                       1.325552 -0.965002   
                          ZTS                        0.441851  0.229762   

                                  adv_20d  dispersion_120d  dispersion_20d  \
date                      asset                                              
2013-01-03 00:00:00+00:00 A      1.397411         0.013270        0.011178   
                          AAL    1.081155         0.013270        0.011178   
2015-12-28 00:00:00+00:00 ZION  -0.724635         0.014916        0.014731   
                          ZTS   -0.180275         0.014916        0.014731   

                                 market_vol_120d  market_vol_20d  \
date                      asset                                    
2013-01-03 00:00:00+00:00 A             0.127654        0.135452   
                          AAL           0.127654        0.135452   
2015-12-28 00:00:00+00:00 ZION          0.183680        0.181479   
                          ZTS           0.183680        0.181479   

                                 volatility_120d  volatility_20d  \
date                      asset                                    
2013-01-03 00:00:00+00:00 A            -0.836546       -1.219809   
                          AAL           1.639924        1.566220   
2015-12-28 00:00:00+00:00 ZION          0.547895        1.099324   
                          ZTS           0.413572       -0.858958   

                                 sector_code_-1  sector_code_0  sector_code_1  \
date                      asset                                                 
2013-01-03 00:00:00+00:00 A                   0              1              0   
                          AAL                 0              0              0   
2015-12-28 00:00:00+00:00 ZION                0              0              0   
                          ZTS                 0              1              0   

                                 sector_code_2  sector_code_3  sector_code_4  \
date                      asset                                                
2013-01-03 00:00:00+00:00 A                  0              0              0   
                          AAL                0              1              0   
2015-12-28 00:00:00+00:00 ZION               0              0              0   
                          ZTS                0              0              0   

                                 sector_code_5  sector_code_6  sector_code_7  \
date                      asset                                                
2013-01-03 00:00:00+00:00 A                  0              0              0   
                          AAL                0              0              0   
2015-12-28 00:00:00+00:00 ZION               1              0              0   
                          ZTS                0              0              0   

                                 sector_code_8

In [15]:
valid_dates = split[split=='valid'].index.get_level_values(0)
valid_start = valid_dates[0].strftime('%Y-%m')
valid_end = valid_dates[-1].strftime('%Y-%m')

In [16]:
test_dates = split[split=='test'].index.get_level_values(0)
test_start = test_dates[0].strftime('%Y-%m-%d')
test_end = test_dates[-1].strftime('%Y-%m-%d')

In [17]:
train_valid_idx = split[(split=='train') | (split=='valid')].index.get_level_values(0).drop_duplicates()

## Market Raw Data

In [18]:
pricing = pd.read_csv(io_utils.interim_path + INFILE3, parse_dates=['date'], index_col='date')
#pricing['date'] = pricing['date'].dt.tz_localize('utc')
#pricing.set_index('date', inplace=True)

In [19]:
volume = pd.read_csv(io_utils.interim_path + INFILE3, parse_dates=['date'], index_col='date')
#volume['date'] = volume['date'].dt.tz_localize('utc')
#volume.set_index('date', inplace=True)

# Prepare Data

## Pricing Data

In [20]:
daily_returns = pricing.pct_change()[1:]

In [21]:
pt_minus1 = pricing.loc[('2015-05-27', 'AAPL')]
pt = pricing.loc[('2015-05-28', 'AAPL')]

In [22]:
assert np.isclose(daily_returns.loc[('2015-05-28', 'AAPL')], (pt-pt_minus1)/pt_minus1)

In [23]:
adv = (pricing*volume).rolling(31, axis=0).mean()
adv = np.clip(adv.iloc[transcost_adv_win:], a_min=transcost_adv_impute, a_max=np.inf)
adv.head(2)

,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,ADM,ADP,ADS,ADSK,AEE,AEP,AES,AET,AFL,AGN,AIG,AIV,AIZ,AJG,AKAM,...,WBA,WDC,WEC,WFC,WHR,WM,WMB,WMT,WRK,WU,WY,WYN,WYNN,XEC,XEL,XL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2011-02-18 00:00:00+00:00,10000.0,10000.0,10000.0,10000.0,NaN,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,...,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,NaN,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,NaN,10000.0,10000.0,10000.0,NaN
2011-02-22 00:00:00+00:00,10000.0,10000.0,10000.0,10000.0,NaN,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,...,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,NaN,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,NaN,10000.0,10000.0,10000.0,NaN


In [24]:
mkt_cap = pricing*volume  # volume referes to daily volume, must use total shares issued

In [25]:
universe_tickers = daily_returns.columns.tolist()
len(universe_tickers)

490

## Alpha Factors

In [31]:
# alphas
alpha_factors = ['Mean_Reversion_Sector_Neutral_Smoothed', 'Momentum_1YR_Smoothed',
       'Overnight_Sentiment_Smoothed', 'avg_combi_alpha', 'ml_alpha']

In [32]:
B_alpha = alpha_fac_df[alpha_factors].copy()
B_alpha.isna().sum()

Mean_Reversion_Sector_Neutral_Smoothed    0
Momentum_1YR_Smoothed                     0
Overnight_Sentiment_Smoothed              0
avg_combi_alpha                           0
ml_alpha                                  0
dtype: int64